In [1]:
# recent gameweek
gameweek = 1

In [2]:
# import basic libraries
import pandas as pd
import numpy as np
from scipy import stats

pd.set_option('max_columns',100)

In [3]:
# check whether team total cost is less than allowed
def is_within_budget(team, cost_threshold):
    return team['now_cost'].sum()/10.0 <= cost_threshold        

In [4]:
# create a custom discrete probability distribution
def discrete_probabilities(nr_elements):
    x = np.arange(nr_elements)
    weights = np.arange(nr_elements,0,-1) / nr_elements
    probabilities = weights / np.sum(weights)
    return stats.rv_discrete(values=(x, probabilities))

In [5]:
# replace a player from a team that is over the budget
def downgrade_team(team,nr_goalkeepers,nr_field):
    # choose a random number between 0-10 with custom weighting
#    custom_probability_generator_team = discrete_probabilities(11)
    custom_ix = custom_probability_generator_team.rvs()
    # find ix of player to be replaced
    replace_ix = team.sort_values(by='valuePoints metric').index[custom_ix]
    # find playing position of player to be replaced
    element_type = team.loc[replace_ix,'element_type']
    # order potential replacements based on valuePoints
    players_ordered = df.loc[(~df.index.isin(team.index)) & (df['element_type']==element_type) & minGames]\
                                                .sort_values(by='valuePoints metric', ascending=False).head(nr_field)
    if element_type > 1:
        custom_probability_generator_new_player = discrete_probabilities(nr_field)
    else:
        custom_probability_generator_new_player = discrete_probabilities(nr_goalkeepers)
    custom_ix = custom_probability_generator_new_player.rvs()
    new_ix = players_ordered.index[custom_ix]
    team = team.drop(replace_ix)
    team = team.append(df.loc[new_ix])
    team = team.sort_values(by='element_type')
    # total cost of dream team
    total_cost = team['now_cost'].sum()/10.0
    # total points for dream team (best player's points doubled for captaincy)
    total_points = team['adjusted points per game'].sum() + team['adjusted points per game'].max()
    return team, total_cost, total_points

In [6]:
def upgrade_team(team, cash_available):
    input_team = team # NOT USED?!
    changes = 0
    for i in range(11):
        player_ix = team.sort_values(by='valuePoints metric').index[i]
        player_cost = df.loc[player_ix,'now_cost'] / 10.0
        element_type = df.loc[player_ix,'element_type']
        better_player_ix = df.loc[(~df.index.isin(team.index)) & (df['element_type']==element_type) & minGames \
                                 & (df['now_cost']/10.0 <= (player_cost+cash_available))]\
                                    .sort_values(by='valuePoints metric', ascending=False).index[0]

        if df.loc[better_player_ix,'valuePoints metric'] > df.loc[player_ix,'valuePoints metric']:
            team = team.drop(player_ix)
            team = df.loc[better_player_ix:better_player_ix].append(team)
            changes = 1

        # total cost of team
        total_cost = team['now_cost'].sum()/10.0
        cash_available = cost_threshold - total_cost
        # total points for dream team (best player's points doubled for captaincy)
        total_points = team['adjusted points per game'].sum() + team['adjusted points per game'].max()  
        team = team.sort_values(by='element_type')
        return team, cash_available, total_cost, total_points, changes

In [7]:
# fetch FPL data
#filepath = '../../data/fpl/data_week' + str(gameweek) + '.csv'
filepath = 'data_week' + str(gameweek) + '.csv'
df = pd.read_csv(filepath, index_col=0)

In [8]:
# value = expected points / cost
df['value'] = df['adjusted points per game'] / (df['now_cost'] / 10.0)
# geometric mean of 'adjusted points per game' and 'value'
df['valuePoints metric'] = np.sqrt(df['adjusted points per game'] * df['value'])

In [9]:
goalkeepers = df['element_type'] == 1
defenders = df['element_type'] == 2
midfielders = df['element_type'] == 3
forwards = df['element_type'] == 4

minGames = df['games played'] >= 5

Choose team formation and find an initial team with the best possible players.

In [19]:
# set up formation (number of defenders, midfielders and forwards, 1 goalkeeper assumed)
formation = [4,4,2]
cost_threshold = 83
# choose the best goalkeeper
team_goalkeeper = df[goalkeepers & minGames].sort_values(by='adjusted points per game', ascending=False).head(1)
# choose formation[0] best defenders
team_defenders = df[defenders & minGames].sort_values(by='adjusted points per game', ascending=False).head(formation[0])
# choose formation[1] best midfielders
team_midfielders = df[midfielders & minGames].sort_values(by='adjusted points per game', ascending=False).head(formation[1])
# choose formation[2] best forwards
team_forwards = df[forwards & minGames].sort_values(by='adjusted points per game', ascending=False).head(formation[2])
# create initial team
team = team_goalkeeper.append(team_defenders).append(team_midfielders).append(team_forwards)
# total cost of dream team
total_cost = team['now_cost'].sum()/10.0
# total points for dream team (best player's points doubled for captaincy)
total_points = team['adjusted points per game'].sum() + team['adjusted points per game'].max()
print('Best team with formation ' + str(formation[0]) + '-' + str(formation[1])+ '-' + str(formation[2]) + ':')
print()
print(team['web_name'])
print()
print('Total cost: ' + str(total_cost))
print()
print('Total points per gameweek: ' + str(total_points))
print()
print('Is this team within budget?')
print(is_within_budget(team, cost_threshold))

#save this team as an initial starting point for searches
dream_team = team

Best team with formation 4-4-2:

id
383              Lloris
259    Alexander-Arnold
255           Robertson
104              Alonso
102         Azpilicueta
254               Salah
302           Fernandes
306            Rashford
272           De Bruyne
224               Vardy
268              Agüero
Name: web_name, dtype: object

Total cost: 96.0

Total points per gameweek: 66.58846393528586

Is this team within budget?
False


Start an iterative process looking for the best team with given formation and budget.

In [20]:
# createa a custom discrete probability distribution for numbers 0-10
custom_probability_generator_team = discrete_probabilities(11)

In [21]:
iterations = 100
nr_goalkeepers = 20
nr_field = 30

best_points = 0
for i in range(iterations):
    team = dream_team
    while ~is_within_budget(team, cost_threshold):
        team, total_cost, total_points = downgrade_team(team,nr_goalkeepers,nr_field)
    
    cash_available = cost_threshold - total_cost
    changes = 1
    while changes==1:
        team, cash_available, total_cost, total_points, changes = upgrade_team(team, cash_available)
    
    if total_points > best_points:
        best_team = team
        best_points = total_points
        best_cost = total_cost
        
    if i%10 == 0:
        print(str(i) + '/' + str(iterations))
    
print(best_team['web_name'])
print()
print('Total cost: ' + str(best_cost))
print()
print('Total points per gameweek: ' + str(best_points))

0/100
10/100
20/100
30/100
40/100
50/100
60/100
70/100
80/100
90/100
id
383         Lloris
102    Azpilicueta
269       Otamendi
232       Chilwell
104         Alonso
468           Jota
254          Salah
302      Fernandes
306       Rashford
437        Antonio
224          Vardy
Name: web_name, dtype: object

Total cost: 83.0

Total points per gameweek: 62.02984376856566


In [22]:
print('Best team with formation ' + str(formation[0]) + '-' + str(formation[1])+ '-' + str(formation[2]) + ':')
print(best_team['web_name'])
print()
print('Total cost: ' + str(best_cost))
print()
print('Total points per gameweek: ' + str(best_points))

Best team with formation 4-4-2:
id
383         Lloris
102    Azpilicueta
269       Otamendi
232       Chilwell
104         Alonso
468           Jota
254          Salah
302      Fernandes
306       Rashford
437        Antonio
224          Vardy
Name: web_name, dtype: object

Total cost: 83.0

Total points per gameweek: 62.02984376856566


In [14]:
print('Best team with formation ' + str(formation[0]) + '-' + str(formation[1])+ '-' + str(formation[2]) + ':')
print(best_team['web_name'])
print()
print('Total cost: ' + str(best_cost))
print()
print('Total points per gameweek: ' + str(best_points))

Best team with formation 3-4-3:
id
383         Lloris
102    Azpilicueta
104         Alonso
255      Robertson
302      Fernandes
272      De Bruyne
306       Rashford
468           Jota
118        Abraham
91            Wood
437        Antonio
Name: web_name, dtype: object

Total cost: 83.0

Total points per gameweek: 61.04238520773331


In [18]:
print('Best team with formation ' + str(formation[0]) + '-' + str(formation[1])+ '-' + str(formation[2]) + ':')
print(best_team['web_name'])
print()
print('Total cost: ' + str(best_cost))
print()
print('Total points per gameweek: ' + str(best_points))

Best team with formation 3-5-2:
id
383         Lloris
102    Azpilicueta
104         Alonso
458           Boly
119        Pulisic
468           Jota
254          Salah
302      Fernandes
306       Rashford
91            Wood
437        Antonio
Name: web_name, dtype: object

Total cost: 83.0

Total points per gameweek: 61.13030414259992
